In [1]:
import pandas as pd
import geopandas as gpd

from shared_utils import rt_utils, catalog_utils, rt_dates, webmap_utils, gtfs_utils_v2

from calitp_data_analysis import get_fs, geography_utils
from segment_speed_utils import helpers, time_series_utils, gtfs_schedule_wrangling, corridor_analysis
from segment_speed_utils.project_vars import SCHED_GCS, SEGMENT_GCS, GTFS_DATA_DICT, COMPILED_CACHED_VIEWS, PROJECT_CRS
import numpy as np
import datetime as dt

ADHOC_PATH = "gs://calitp-analytics-data/data-analyses/rt_segment_speeds/adhoc/"

In [2]:
target_mph = 16
target_mps = target_mph / rt_utils.MPH_PER_MPS

In [3]:
SEGMENT_GCS

'gs://calitp-analytics-data/data-analyses/rt_segment_speeds/'

In [4]:
from siuba import *

In [5]:
import gcsfs
import google.auth
credentials, project = google.auth.default()
fs = gcsfs.GCSFileSystem(token=credentials)

# Lincoln Corridor Speed History

* support D7 Lincoln Bus Lane Feasibility Study by illustrating worsening speeds and bus rider delay over the last few years
* will refactor into a script pending speedmap segments work (goal is to align on those)

In [6]:
# analysis_date = rt_dates.DATES['mar2023']
# analysis_date = rt_dates.DATES['sep2022a']

## refactor to use corridor analysis tooling

In [7]:
analysis_date = rt_dates.DATES['mar2023']

In [8]:
segment_speeds = corridor_analysis.import_speedmap_segment_speeds(analysis_date)

frequencies = corridor_analysis.get_max_frequencies(segment_speeds)

trip_speeds = corridor_analysis.import_trip_speeds(analysis_date)

/home/jovyan/data-analyses/rt_segment_speeds/segment_speed_utils/corridor_analysis.py:33: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  group_cols + ['direction_id']).sum().groupby(


2.0 percent of segments have no speed


In [9]:
# corr = pd.read_excel(f'{ADHOC_PATH}corr_hs.xlsx', sheet_name='Corridor Input')
# corr = corr.astype({'start_segment_id': str, 'end_segment_id': str, 'shape_id':str})

In [10]:
# row = corr.loc[21]

In [11]:
# corr_gdf = corridor_analysis.corridor_from_segments(speed_segments_gdf=segment_speeds,
#                                                organization_source_record_id=row.organization_source_record_id,
#                                                shape_id=row.shape_id,
#                                                start_seg_id=row.start_segment_id,
#                                                end_seg_id=row.end_segment_id)

In [12]:
# corr_gdf.to_parquet(f'{ADHOC_PATH}lincoln_corridor.parquet', filesystem=fs)

In [13]:
corr_gdf = gpd.read_parquet(f'{ADHOC_PATH}lincoln_corridor.parquet', storage_options = {"token": credentials.token})

In [14]:
corridor_trips = corridor_analysis.find_corridor_data(segment_speeds, corr_gdf, trip_speeds,
                                                     custom_relevance_meters=5000
                                                     )

In [15]:
if 'route_short_name' not in corridor_trips.columns:

    trips = helpers.import_scheduled_trips(analysis_date, columns=['gtfs_dataset_key', 'route_id', 'route_short_name'])
    trips = trips.rename(columns={'gtfs_dataset_key':'schedule_gtfs_dataset_key'}).drop_duplicates()
    corridor_trips = corridor_trips.merge(trips, on =['schedule_gtfs_dataset_key', 'route_id'])

In [16]:
corridor_results = corridor_analysis.analyze_corridor_trips(corridor_trips)

0.0 percent of trips with zero seconds
0.6 percent of trips with speeds > 80mph or < 3mph


In [17]:
# corridor_analysis.validate_corridor_routes(corr_gdf, corridor_trips)

In [18]:
corridor_improvements = corridor_analysis.analyze_corridor_improvements(corridor_results, **{'trip_mph_floor': 16, 'trip_percent_speedup': 15})

median: 13.36mph... mph floor mode: 16mph


In [19]:
summarized = corridor_analysis.summarize_corridor_improvements(corridor_improvements, frequencies, corr_gdf)

In [20]:
summarized = summarized.assign(delay_per_trip = summarized.delay_minutes / summarized.n_trips_daily,
                              analysis_date = analysis_date)

In [21]:
summarized

,corridor_id,schedule_gtfs_dataset_key,intervention_assumption,corridor_miles,corridor_seconds,improved_corridor_seconds,delay_seconds,delay_minutes,minutes_per_mile,route_short_name,route_id,median_corridor_mph,n_trips_daily,trips_hr_sch,trips_per_hr_peak_directional,delay_per_trip,analysis_date
0,584447099936112567,dbbe8ee4864a2715a40749605395d584,trips achieve 16mph or existing spd if higher,4.5,206462.0,163322.1,43139.9,719.0,160.3,"[3, R3]","[3556, 3570]",13.4,155,"[4.7, 1.7]",6.4,4.63871,2023-03-15


In [22]:
def new_lincoln(analysis_date, corr_gdf = corr_gdf):
    
    try:
        segment_speeds = corridor_analysis.import_speedmap_segment_speeds(analysis_date)
        frequencies = corridor_analysis.get_max_frequencies(segment_speeds)
        trip_speeds = corridor_analysis.import_trip_speeds(analysis_date)
        corridor_trips = corridor_analysis.find_corridor_data(segment_speeds, corr_gdf, trip_speeds,
                                                         custom_relevance_meters=5000
                                                         )
        if 'route_short_name' not in corridor_trips.columns:

            trips = helpers.import_scheduled_trips(analysis_date, columns=['gtfs_dataset_key', 'route_id', 'route_short_name'])
            trips = trips.rename(columns={'gtfs_dataset_key':'schedule_gtfs_dataset_key'}).drop_duplicates()
            corridor_trips = corridor_trips.merge(trips, on =['schedule_gtfs_dataset_key', 'route_id'])

        corridor_results = corridor_analysis.analyze_corridor_trips(corridor_trips)
        corridor_improvements = corridor_analysis.analyze_corridor_improvements(corridor_results, **{'trip_mph_floor': 16, 'trip_percent_speedup': 15})
        summarized = corridor_analysis.summarize_corridor_improvements(corridor_improvements, frequencies, corr_gdf)
        summarized = summarized.assign(delay_per_trip = summarized.delay_minutes / summarized.n_trips_daily,
                                  analysis_date = analysis_date)
        return summarized
    
    except Exception as e:
        print(f'{analysis_date} failed')
        print(e)
        return

In [23]:
all_summaries = []

In [ ]:
for key in rt_dates.DATES.keys():
    date = rt_dates.DATES[key]
    print(date)
    all_summaries += [new_lincoln(date)]

2023-03-15


/home/jovyan/data-analyses/rt_segment_speeds/segment_speed_utils/corridor_analysis.py:33: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  group_cols + ['direction_id']).sum().groupby(


2.0 percent of segments have no speed
0.0 percent of trips with zero seconds
0.6 percent of trips with speeds > 80mph or < 3mph
median: 13.36mph... mph floor mode: 16mph
2023-04-10


/home/jovyan/data-analyses/rt_segment_speeds/segment_speed_utils/corridor_analysis.py:33: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  group_cols + ['direction_id']).sum().groupby(


2.0 percent of segments have no speed
0.0 percent of trips with zero seconds
1.3 percent of trips with speeds > 80mph or < 3mph
median: 13.34mph... mph floor mode: 16mph
2023-04-11


...

1.9 percent of segments have no speed
0.9 percent of trips with zero seconds
0.0 percent of trips with speeds > 80mph or < 3mph
median: 12.71mph... mph floor mode: 16mph
2024-05-22


/home/jovyan/data-analyses/rt_segment_speeds/segment_speed_utils/corridor_analysis.py:33: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  group_cols + ['direction_id']).sum().groupby(


2.0 percent of segments have no speed
0.0 percent of trips with zero seconds
0.0 percent of trips with speeds > 80mph or < 3mph
median: 11.81mph... mph floor mode: 16mph
2024-06-12


/home/jovyan/data-analyses/rt_segment_speeds/segment_speed_utils/corridor_analysis.py:33: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  group_cols + ['direction_id']).sum().groupby(


1.8 percent of segments have no speed
2024-06-12 failed
division by zero
2024-07-17


In [25]:
all_df = pd.concat(all_summaries)

In [26]:
import altair as alt

In [27]:
all_df.analysis_date = all_df.analysis_date.apply(lambda x: pd.Timestamp(x))

In [28]:
chart = alt.Chart(all_df).mark_point().encode(
    x=alt.X('analysis_date:T', axis = alt.Axis(title = 'Date'.upper(), format = ("%b %Y"))),
    y='delay_per_trip'
).properties(width=600, height=400)

chart + chart.transform_regression('analysis_date', 'delay_per_trip').mark_line()

alt.LayerChart(...)

## old

In [114]:
def get_bbb_feed(analysis_date):

    feeds = gtfs_utils_v2.schedule_daily_feed_to_gtfs_dataset_name(analysis_date)
    bbb_feed = feeds >> filter(_.name.str.contains('Big Blue'))
    return bbb_feed

# bbb_dataset_key = bbb_feed.gtfs_dataset_key.iloc[0]
# bbb_feed_key = bbb_feed.feed_key.iloc[0]

In [115]:
# mar23 = pd.read_parquet(f'{SEGMENT_GCS}speeds_stop_segments_{analysis_date}.parquet')

In [116]:
# mar23_shape = pd.read_parquet(f'{SEGMENT_GCS}segment_options/shape_stop_segments_{analysis_date}.parquet')

In [117]:
def get_shapes(analysis_date, bbb_feed):
    shapes = pd.read_parquet(f'{SEGMENT_GCS}segment_options/shape_stop_segments_{analysis_date}.parquet')
    bbb_shapes = shapes >> filter(_.schedule_gtfs_dataset_key == bbb_feed.gtfs_dataset_key.iloc[0])
    bbb_shapes.geometry = gpd.GeoSeries.from_wkb(bbb_shapes.geometry)
    bbb_shapes = gpd.GeoDataFrame(bbb_shapes, geometry=bbb_shapes.geometry, crs=PROJECT_CRS)
    return bbb_shapes

In [118]:
def get_speeds(analysis_date, bbb_feed):
    speeds = pd.read_parquet(f'{SEGMENT_GCS}speeds_stop_segments_{analysis_date}.parquet')
    bbb_speeds = (speeds >> filter(_.schedule_gtfs_dataset_key == bbb_feed.gtfs_dataset_key.iloc[0])
                     >> select(_.speed_mph, _.meters_elapsed, _.sec_elapsed, _.trip_instance_key, _.stop_sequence)
                 )
    return bbb_speeds

In [121]:
bbb_feed = get_bbb_feed(analysis_date)

bbb_speeds = get_speeds(analysis_date, bbb_feed)

bbb_shapes = get_shapes(analysis_date, bbb_feed)

/tmp/ipykernel_1841/1351526470.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  bbb_shapes.geometry = gpd.GeoSeries.from_wkb(bbb_shapes.geometry)


In [122]:
lincoln_trips = gtfs_utils_v2.get_trips(selected_date=analysis_date,
                                                 operator_feeds=[bbb_feed.feed_key.iloc[0]],
                                                custom_filtering = {'route_short_name': ['3', 'R3']})
lincoln_trips = lincoln_trips >> select(_.trip_instance_key, _.route_short_name, _.trip_first_departure_ts)

lincoln_shapes = bbb_shapes >> inner_join(_, lincoln_trips, on = ['trip_instance_key'])

lincoln_shapes = lincoln_shapes >> inner_join(_, bbb_speeds, on=['trip_instance_key', 'stop_sequence'])

In [123]:
lincoln_shapes >> count(_.trip_instance_key)

,trip_instance_key,n
0,012d2b7d1e5d77757ebb2414bbeedfae,35
1,05336b499070f87c7f6b4c8cf1c9172a,34
2,0628c213593d3b31d6ca00d1e4b81062,34
3,06ca5ebc8bc10cc28f1ec25b7e8a1d99,33
4,08c415d5ca6ef9453381b97dcc2c1b58,14
...,...,...
151,fd9f001bcb7ab00019f453a72fdf563c,36
152,fdcf0f29931f5d955bbb88c6cc8f5423,36
153,fe4071eea14b27590e904a88190b328b,36
154,ff0a5068f000cf3c2edffb2f838699a2,34


In [149]:
lincoln_shapes

,trip_instance_key,shape_array_key,stop_id1,stop_sequence,geometry,stop_id2,segment_id,stop_pair,schedule_gtfs_dataset_key,route_id,direction_id,st_trip_instance_key,segment_uuid,route_short_name,trip_first_departure_ts,speed_mph,meters_elapsed,sec_elapsed,target_sec_elapsed
0,012d2b7d1e5d77757ebb2414bbeedfae,95fcdc2f2f2e180e04129bfd4829a239,529,2,"LINESTRING (149930.366 -452551.442, 149930.311...",532,529-532-1,529__532,dbbe8ee4864a2715a40749605395d584,3556,0.0,012d2b7d1e5d77757ebb2414bbeedfae,dbbe8ee4864a2715a40749605395d584__3556__0.0__5...,3,2023-03-15 15:09:00+00:00,32.722493,1623.691,111.0,227.012298
1,012d2b7d1e5d77757ebb2414bbeedfae,95fcdc2f2f2e180e04129bfd4829a239,532,3,"LINESTRING (149889.542 -450921.566, 149878.542...",1668,532-1668-1,532__1668,dbbe8ee4864a2715a40749605395d584,3556,0.0,012d2b7d1e5d77757ebb2414bbeedfae,dbbe8ee4864a2715a40749605395d584__3556__0.0__5...,3,2023-03-15 15:09:00+00:00,15.994270,343.194,48.0,47.982811
2,012d2b7d1e5d77757ebb2414bbeedfae,95fcdc2f2f2e180e04129bfd4829a239,1668,4,"LINESTRING (149758.895 -450797.226, 149757.478...",1670,1668-1670-1,1668__1670,dbbe8ee4864a2715a40749605395d584,3556,0.0,012d2b7d1e5d77757ebb2414bbeedfae,dbbe8ee4864a2715a40749605395d584__3556__0.0__1...,3,2023-03-15 15:09:00+00:00,26.448160,484.745,41.0,67.773410
3,012d2b7d1e5d77757ebb2414bbeedfae,95fcdc2f2f2e180e04129bfd4829a239,1670,5,"LINESTRING (149202.291 -450737.943, 149131.117...",1677,1670-1677-1,1670__1677,dbbe8ee4864a2715a40749605395d584,3556,0.0,012d2b7d1e5d77757ebb2414bbeedfae,dbbe8ee4864a2715a40749605395d584__3556__0.0__1...,3,2023-03-15 15:09:00+00:00,12.720001,454.895,80.0,63.600007
4,012d2b7d1e5d77757ebb2414bbeedfae,95fcdc2f2f2e180e04129bfd4829a239,1677,6,"LINESTRING (148866.647 -450742.541, 148817.658...",1636,1677-1636-1,1677__1636,dbbe8ee4864a2715a40749605395d584,3556,0.0,012d2b7d1e5d77757ebb2414bbeedfae,dbbe8ee4864a2715a40749605395d584__3556__0.0__1...,3,2023-03-15 15:09:00+00:00,32.861482,690.429,47.0,96.530605
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5205,e22e78fa929cd7c394696b8f7a7cfe7e,71e4b27513ed17a85f58ed99891d3ace,1186,5,"LINESTRING (140174.335 -443728.422, 140196.765...",51,1186-51-1,1186__51,dbbe8ee4864a2715a40749605395d584,3570,1.0,2b4c0e5b83cd2e106253084aea0f37b2,dbbe8ee4864a2715a40749605395d584__3570__1.0__1...,R3,2023-03-15 22:10:00+00:00,6.580528,870.736,296.0,121.739777
5206,e22e78fa929cd7c394696b8f7a7cfe7e,71e4b27513ed17a85f58ed99891d3ace,51,6,"LINESTRING (140802.700 -444313.078, 140812.079...",598,51-598-1,51__598,dbbe8ee4864a2715a40749605395d584,3570,1.0,2b4c0e5b83cd2e106253084aea0f37b2,dbbe8ee4864a2715a40749605395d584__3570__1.0__5...,R3,2023-03-15 22:10:00+00:00,6.585570,959.721,326.0,134.180992
5207,e22e78fa929cd7c394696b8f7a7cfe7e,71e4b27513ed17a85f58ed99891d3ace,598,7,"LINESTRING (141590.981 -444902.647, 141646.001...",1201,598-1201-1,598__1201,dbbe8ee4864a2715a40749605395d584,3570,1.0,2b4c0e5b83cd2e106253084aea0f37b2,dbbe8ee4864a2715a40749605395d584__3570__1.0__5...,R3,2023-03-15 22:10:00+00:00,6.589481,621.538,211.0,86.898782
5208,e22e78fa929cd7c394696b8f7a7cfe7e,71e4b27513ed17a85f58ed99891d3ace,1201,8,"LINESTRING (142119.678 -445199.270, 142172.928...",476,1201-476-1,1201__476,dbbe8ee4864a2715a40749605395d584,3570,1.0,2b4c0e5b83cd2e106253084aea0f37b2,dbbe8ee4864a2715a40749605395d584__3570__1.0__1...,R3,2023-03-15 22:10:00+00:00,6.560909,926.798,316.0,129.577945


In [124]:
lincoln_shapes['target_sec_elapsed'] = lincoln_shapes.meters_elapsed / target_mps

assert lincoln_shapes.crs == corr_gdf.crs

lincoln_clipped = gpd.clip(lincoln_shapes, corr_gdf)

lincoln_clipped['speed_delay'] = (lincoln_clipped.sec_elapsed - lincoln_clipped.target_sec_elapsed).clip(lower=0)

In [125]:
# (lincoln_clipped >> select(-_.trip_first_departure_ts)
#     >> filter(_.trip_instance_key == '08c415d5ca6ef9453381b97dcc2c1b58')
# ).explore()

In [126]:
lincoln_clipped.speed_delay.sum() / 60

1546.4925773156249

In [127]:
def lincoln_corridor_delay(analysis_date):
    
    # print(analysis_date)
    bbb_feed = get_bbb_feed(analysis_date)
    bbb_speeds = get_speeds(analysis_date, bbb_feed)
    bbb_shapes = get_shapes(analysis_date, bbb_feed)
    
    lincoln_trips = gtfs_utils_v2.get_trips(selected_date=analysis_date,
                                                 operator_feeds=[bbb_feed.feed_key.iloc[0]],
                                                custom_filtering = {'route_short_name': ['3', 'R3']})
    lincoln_trips = lincoln_trips >> select(_.trip_instance_key, _.route_short_name, _.trip_first_departure_ts)
    sched_trip_count = lincoln_trips.shape[0]

    lincoln_shapes = bbb_shapes >> inner_join(_, lincoln_trips, on = ['trip_instance_key'])

    lincoln_shapes = lincoln_shapes >> inner_join(_, bbb_speeds, on=['trip_instance_key', 'stop_sequence'])
    print((lincoln_shapes >> count(_.trip_instance_key)).shape)
    assert not lincoln_shapes.empty
    rt_trip_count = (lincoln_shapes >> count(_.trip_instance_key)).shape[0]
    
    lincoln_shapes['target_sec_elapsed'] = lincoln_shapes.meters_elapsed / target_mps
    assert lincoln_shapes.crs == corr_gdf.crs
    lincoln_clipped = gpd.clip(lincoln_shapes, corr_gdf)
    lincoln_clipped['speed_delay'] = (lincoln_clipped.sec_elapsed - lincoln_clipped.target_sec_elapsed).clip(lower=0)
    delay_minutes = lincoln_clipped.speed_delay.sum() / 60
    
    return (analysis_date, delay_minutes, rt_trip_count, rt_trip_count / sched_trip_count)

In [128]:
lincoln_corridor_delay(analysis_date)

/tmp/ipykernel_1841/1351526470.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  bbb_shapes.geometry = gpd.GeoSeries.from_wkb(bbb_shapes.geometry)


(156, 2)


('2023-03-15', 1546.4925773156249, 156, 0.9629629629629629)

In [135]:
test = list(rt_dates.DATES.keys())[:5]

In [136]:
test

['mar2023', 'apr2023a', 'apr2023b', 'apr2023', 'apr2023c']

In [22]:
def compile_lincoln_delays():
    lincoln_delays = []
    for date in rt_dates.DATES.keys():
    # for date in test:
        analysis_date = rt_dates.DATES[date]
        print(analysis_date)
        try:
            lincoln_delays += [lincoln_corridor_delay(analysis_date)]
        except:
            print(f'failed for {analysis_date}')
    # return lincoln_delays
    lincoln_delay_df = pd.DataFrame(lincoln_delays,
                                    columns=['date_str', 'delay_minutes', 'vp_trip_count',
                                             'vp_pct_sched'])
    lincoln_delay_df['date'] = lincoln_delay_df.date_str.apply(lambda x: dt.datetime.fromisoformat(x).date())
    lincoln_delay_df['weekday'] = lincoln_delay_df.date.apply(lambda x: x.strftime('%a'))
    # df = df >> filter(-_.weekday.isin(['Sat', 'Sun', 'Mon', 'Fri']))
    return lincoln_delay_df

In [23]:
lincoln_delay_df = compile_lincoln_delays()

2022-02-08
failed for 2022-02-08
2022-03-30
failed for 2022-03-30
2022-05-04
failed for 2022-05-04
2022-06-15
failed for 2022-06-15
2022-07-13
failed for 2022-07-13
2022-08-17
failed for 2022-08-17
2022-09-14
failed for 2022-09-14
2022-09-21


/tmp/ipykernel_378/1351526470.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  bbb_shapes.geometry = gpd.GeoSeries.from_wkb(bbb_shapes.geometry)


(182, 2)
2022-10-12
failed for 2022-10-12
2022-11-07
failed for 2022-11-07
2022-11-08
failed for 2022-11-08
2022-11-09
failed for 2022-11-09
2022-11-10
failed for 2022-11-10
2022-11-16
failed for 2022-11-16
2022-12-14
failed for 2022-12-14
2023-01-18
failed for 2023-01-18
2023-02-15
failed for 2023-02-15
2023-03-15


/tmp/ipykernel_378/1351526470.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  bbb_shapes.geometry = gpd.GeoSeries.from_wkb(bbb_shapes.geometry)


(156, 2)
2023-04-10


...

(123, 2)
2024-04-20


/tmp/ipykernel_378/1351526470.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  bbb_shapes.geometry = gpd.GeoSeries.from_wkb(bbb_shapes.geometry)


(120, 2)
2024-04-21


/tmp/ipykernel_378/1351526470.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  bbb_shapes.geometry = gpd.GeoSeries.from_wkb(bbb_shapes.geometry)


(107, 2)


In [24]:
lincoln_delay_df = lincoln_delay_df >> filter(_.vp_pct_sched > .8)

In [25]:
lincoln_delay_df['delay_per_trip'] = lincoln_delay_df.delay_minutes / lincoln_delay_df.vp_trip_count

In [26]:
lincoln_delay_df = lincoln_delay_df >> filter(-_.weekday.isin(['Sat', 'Sun', 'Mon', 'Fri']))

In [27]:
lincoln_delay_df

,date_str,delay_minutes,vp_trip_count,vp_pct_sched,date,weekday,delay_per_trip
0,2022-09-21,1414.623417,182,0.928571,2022-09-21,Wed,7.772656
1,2023-03-15,1237.075534,156,0.962963,2023-03-15,Wed,7.929971
3,2023-04-11,1254.466843,145,0.895062,2023-04-11,Tue,8.651495
4,2023-04-12,1234.140576,160,0.987654,2023-04-12,Wed,7.713379
5,2023-04-13,1090.717845,146,0.901235,2023-04-13,Thu,7.470670
9,2023-05-17,1276.081033,144,0.888889,2023-05-17,Wed,8.861674
10,2023-06-14,1206.520870,144,0.888889,2023-06-14,Wed,8.378617
11,2023-07-12,1302.071853,162,1.000000,2023-07-12,Wed,8.037481
12,2023-08-15,1508.928380,160,0.987654,2023-08-15,Tue,9.430802
13,2023-09-13,1297.524248,147,0.907407,2023-09-13,Wed,8.826696


In [28]:
import altair as alt

In [29]:
lincoln_delay_df.date = lincoln_delay_df.date.apply(lambda x: pd.Timestamp(x))

In [30]:
chart = alt.Chart(lincoln_delay_df).mark_point().encode(
    x=alt.X('date:T', axis = alt.Axis(title = 'Date'.upper(), format = ("%b %Y"))),
    y='delay_per_trip'
).properties(width=600, height=400)

chart + chart.transform_regression('date', 'delay_per_trip').mark_line()

alt.LayerChart(...)

In [31]:
lincoln_delay_df.to_csv('lincoln_delays_preliminary.csv')

In [32]:
(10 - 8) / 8

0.25